In [34]:
                                        ###    Upload Neo4J Data   ###

import pandas as pd
from neo4j import GraphDatabase

# Cambiar por tus credenciales
URI = "bolt://neo4j:7687"
USER = "neo4j"
PASSWORD = "longerpass123"

driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))

def cargar_datos_a_neo4j():
    # Leer archivos CSV
    proveedores = pd.read_csv('proveedor.csv', sep=';')
    telefonos = pd.read_csv('telefono.csv', sep=';')
    pedidos = pd.read_csv('op.csv', sep=';')
    productos = pd.read_csv('producto.csv', sep=';')
    detalle_op = pd.read_csv('detalle_op.csv', sep=';')

    with driver.session() as session:
        # Crear proveedores
        for _, row in proveedores.iterrows():
            session.run("""
                MERGE (p:Proveedor {id_proveedor: $id})
                SET p.CUIT = $CUIT, p.razon_social = $razon, p.tipo_sociedad = $sociedad,
                    p.direccion = $dir, p.activo = $activo, p.habilitado = $habilitado
            """, id=row['id_proveedor'], CUIT=row['CUIT_proveedor'], razon=row['razon_social'],
                 sociedad=row['tipo_sociedad'], dir=row['direccion'],
                 activo=bool(row['activo']), habilitado=bool(row['habilitado']))

        # Crear telefonos y relaciones
        for _, row in telefonos.iterrows():
            session.run("""
                MERGE (t:Telefono {codigo_area: $area, nro_telefono: $nro})
                SET t.tipo = $tipo
                WITH t
                MATCH (p:Proveedor {id_proveedor: $id_prov})
                MERGE (p)-[:TIENE_TELEFONO]->(t)
            """, area=row['codigo_area'], nro=row['nro_telefono'], tipo=row['tipo'],
                 id_prov=row['id_proveedor'])

        # Crear pedidos y relaciones
        for _, row in pedidos.iterrows():
            session.run("""
                MERGE (o:Pedido {id_pedido: $id})
                SET o.fecha = $fecha, o.total_sin_iva = $total, o.iva = $iva
                WITH o
                MATCH (p:Proveedor {id_proveedor: $id_prov})
                MERGE (o)-[:ES_DE]->(p)
            """, id=row['id_pedido'], fecha=row['fecha'], total=row['total_sin_iva'],
                 iva=row['iva'], id_prov=row['id_proveedor'])

        # Crear productos
        for _, row in productos.iterrows():
            session.run("""
                MERGE (pr:Producto {id_producto: $id})
                SET pr.descripcion = $desc, pr.marca = $marca, pr.categoria = $cat,
                    pr.precio = $precio, pr.stock_actual = $stock, pr.stock_futuro = $futuro
            """, id=row['id_producto'], desc=row['descripcion'], marca=row['marca'],
                 cat=row['categoria'], precio=row['precio'],
                 stock=row['stock_actual'], futuro=row['stock_futuro'])

        # Crear relaciones Pedido -> Producto
        for _, row in detalle_op.iterrows():
            session.run("""
                MATCH (o:Pedido {id_pedido: $id_pedido})
                MATCH (pr:Producto {id_producto: $id_prod})
                MERGE (o)-[r:INCLUYE {nro_item: $item, cantidad: $cantidad}]->(pr)
            """, id_pedido=row['id_pedido'], id_prod=row['id_producto'],
                 item=row['nro_item'], cantidad=row['cantidad'])

    print("Datos cargados correctamente a Neo4j.")


cargar_datos_a_neo4j()


Datos cargados correctamente a Neo4j.


In [39]:
                                        ###    Upload MongoDB Data   ###

import pandas as pd
from pymongo import MongoClient
from datetime import datetime

# Conectarse a MongoDB
client = MongoClient("mongodb://mongo:27017")
db = client["mi_base"]
col_proveedores = db["proveedores"]
col_productos = db["productos"]
col_pedidos = db["pedidos"]

# Cargar CSVs
proveedores = pd.read_csv("proveedor.csv", sep=';')
telefonos = pd.read_csv("telefono.csv", sep=';')
productos = pd.read_csv("producto.csv", sep=';')
pedidos = pd.read_csv("op.csv", sep=';')
detalle_op = pd.read_csv("detalle_op.csv", sep=';')

# Crear colección de proveedores (embebiendo teléfonos)
for _, prov in proveedores.iterrows():
    tel_prov = telefonos[telefonos["id_proveedor"] == prov["id_proveedor"]]
    telefonos_embebidos = [
        {
            "codigo_area": int(t["codigo_area"]),
            "nro_telefono": str(t["nro_telefono"]),
            "tipo": t["tipo"]
        }
        for _, t in tel_prov.iterrows()
    ]
    col_proveedores.insert_one({
        "_id": int(prov["id_proveedor"]),
        "CUIT": prov["CUIT_proveedor"],
        "razon_social": prov["razon_social"],
        "tipo_sociedad": prov["tipo_sociedad"],
        "direccion": prov["direccion"],
        "activo": bool(prov["activo"]),
        "habilitado": bool(prov["habilitado"]),
        "telefonos": telefonos_embebidos
    })

# Crear colección de productos
for _, prod in productos.iterrows():
    col_productos.insert_one({
        "_id": int(prod["id_producto"]),
        "descripcion": prod["descripcion"],
        "marca": prod["marca"],
        "categoria": prod["categoria"],
        "precio": float(prod["precio"]),
        "stock_actual": int(prod["stock_actual"]),
        "stock_futuro": int(prod["stock_futuro"])
    })

# Crear colección de pedidos (embebiendo detalle_op)
for _, op in pedidos.iterrows():
    detalles = detalle_op[detalle_op["id_pedido"] == op["id_pedido"]]
    detalle_embebido = [
        {
            "id_producto": int(det["id_producto"]),
            "nro_item": int(det["nro_item"]),
            "cantidad": float(det["cantidad"])
        }
        for _, det in detalles.iterrows()
    ]
    col_pedidos.insert_one({
        "_id": int(op["id_pedido"]),
        "id_proveedor": int(op["id_proveedor"]),
        "fecha": datetime.strptime(op["fecha"], "%d/%m/%Y"),
        "total_sin_iva": float(op["total_sin_iva"]),
        "iva": float(op["iva"]),
        "detalle": detalle_embebido
    })

print("Datos cargados exitosamente en MongoDB.")

"""
for prov in col_proveedores.find():
    print(f"Proveedor: {prov['razon_social']}")
    for tel in prov.get('telefonos', []):
        print(f"  Teléfono: ({tel['codigo_area']}) {tel['nro_telefono']} - Tipo: {tel['tipo']}")

for prod in col_productos.find():
    print(f"Producto: {prod['descripcion']} - Marca: {prod['marca']} - Precio: ${prod['precio']}")


for pedido in col_pedidos.find():
    print(f"\nPedido ID: {pedido['_id']} - Fecha: {pedido['fecha'].strftime('%d/%m/%Y')} - Total sin IVA: {pedido['total_sin_iva']}")
    print(f"Proveedor ID: {pedido['id_proveedor']}")
    for item in pedido.get('detalle', []):
        print(f"  Producto ID: {item['id_producto']} - Ítem: {item['nro_item']} - Cantidad: {item['cantidad']}")


print("Proveedores:", col_proveedores.count_documents({}))
print("Productos:", col_productos.count_documents({}))
print("Pedidos:", col_pedidos.count_documents({}))

"""


Proveedor: Arcos Plateados
  Teléfono: (513) 4998612 - Tipo: M
  Teléfono: (992) 4241515 - Tipo: F
Proveedor: Grupo Fern�ndez
  Teléfono: (638) 4819478 - Tipo: F
Proveedor: Innovation
  Teléfono: (915) 4686947 - Tipo: F
Proveedor: Alimentos Naturales
Proveedor: Servicios Integrales
  Teléfono: (662) 4466147 - Tipo: M
Proveedor: Transporte R�pido
Proveedor: Productos de Limpieza
  Teléfono: (158) 4583134 - Tipo: M
  Teléfono: (351) 4268273 - Tipo: M
Proveedor: Energ�a Renovable
  Teléfono: (132) 4962172 - Tipo: F
  Teléfono: (212) 4515936 - Tipo: F
  Teléfono: (629) 4699941 - Tipo: F
Proveedor: Estes
  Teléfono: (172) 4344579 - Tipo: M
Proveedor: Saunders
  Teléfono: (189) 4229129 - Tipo: F
Proveedor: Alvarado
  Teléfono: (115) 4573879 - Tipo: F
Proveedor: Hubbard
  Teléfono: (146) 4579375 - Tipo: F
  Teléfono: (336) 4869818 - Tipo: M
  Teléfono: (698) 4653876 - Tipo: F
  Teléfono: (712) 4652912 - Tipo: F
Proveedor: Knowles
  Teléfono: (491) 4639673 - Tipo: F
  Teléfono: (694) 4485437 -